In [1]:
import zipfile
#import wget
from matplotlib import pyplot as plt
from collections import Counter
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from gensim.models import KeyedVectors
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [ ]:
'''
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
filename = wget.download(url=url)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(path="./")
'''

'\nurl = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"\nfilename = wget.download(url=url)\nwith zipfile.ZipFile(filename, \'r\') as zip_ref:\n    zip_ref.extractall(path="./")\n'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [180]:
!unzip 'drive/MyDrive/crawl-300d-2M.vec.zip'

Archive:  drive/MyDrive/crawl-300d-2M.vec.zip
  inflating: crawl-300d-2M.vec       


In [ ]:
embeddings_index = KeyedVectors.load_word2vec_format('crawl-300d-2M.vec',
                                                     binary=False)

In [5]:
with open("drive/MyDrive/Sherlock.txt", 'r', encoding='utf-8') as file:
    book = file.read()


In [ ]:
book[0:10000]

'\ufeffThe Project Gutenberg eBook of The Adventures of Sherlock Holmes\n    \nThis ebook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this ebook or online\nat www.gutenberg.org. If you are not located in the United States,\nyou will have to check the laws of the country where you are located\nbefore using this eBook.\n\nTitle: The Adventures of Sherlock Holmes\n\n\nAuthor: Arthur Conan Doyle\n\nRelease date: March 1, 1999 [eBook #1661]\n                Most recently updated: October 10, 2023\n\nLanguage: English\n\nCredits: an anonymous Project Gutenberg volunteer and Jose Menendez\n\n\n*** START OF THE PROJECT GUTENBERG EBOOK THE ADVENTURES OF SHERLOCK HOLMES ***\n\n\n\n\nThe Adventures of Sherlock Holmes\n\nby Arthur Conan Doyle\n\n\nContents\n\n   I.     A Scandal in Bohemia\n 

In [154]:
paragraphs = book.split('. ')

In [39]:
len(paragraphs)

3984

In [40]:
paragraphs[1000]

'I think that the change would do you good, and you\nare always so interested in Mr'

In [41]:
for i, paragraph in enumerate(paragraphs):
    paragraphs[i] = '<SOS> ' + paragraph
    paragraphs[i] += ' <EOS>'

In [104]:
paragraphs[1000]

'<SOS> I think that the change would do you good, and you\nare always so interested in Mr <EOS>'

In [107]:
paragraphs[1000][0:500]

'<SOS> I think that the change would do you good, and you\nare always so interested in Mr <EOS>'

In [43]:
token = Tokenizer(num_words=10000,
                  filters='!"#$%&()*+,-./:;=?@[\\]^_`{|}~\t\n', lower=True,
                  split=' ', char_level=False, oov_token=None,
                  document_count=0)
token.fit_on_texts(paragraphs)

In [44]:
sequences = token.texts_to_sequences(paragraphs)

In [155]:
sequences[1111]

[2,
 7,
 3991,
 5882,
 135,
 60,
 5883,
 4,
 3857,
 238,
 14,
 355,
 25,
 79,
 34,
 1,
 2477,
 3]

In [176]:
def create_dataset(sequences, window = 10, overlap = 0.5):
    inputs = []
    outputs = []
    for seq in sequences:
        for i in range(0, len(seq)-1, int(window*overlap)):
            inputs.append(seq[i:i+window])
            outputs.append(seq[i+1:i+window+1])

    return inputs, outputs

In [ ]:
max_len = 10

x, y = create_dataset(sequences, max_len, 0.5)

In [172]:
x[1005]

[3516, 3]

In [173]:
y[1059]

[50, 14, 173, 3526, 1, 106, 10, 18, 227, 59]

In [ ]:
def encode_text(inputs,chars, window_size):
    num_chars = len(chars)
    chars_to_indices = dict((c, i) for i, c in enumerate(chars))

In [178]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dropout, Dense, Input, Concatenate, Dot, RepeatVector, TimeDistributed, Multiply, Lambda, Flatten, Activation, Reshape, BatchNormalization, LSTM
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model

In [ ]:
lengths=[len(seq) for seq in x]
lengths=dict(Counter(lengths))
train_sequences = pad_sequences(x, maxlen=max_len)
train_sequences_y = pad_sequences(y, maxlen=max_len)
reverse_dictionary = token.index_word
dictionary = dict([(value, key) for (key, value) in reverse_dictionary.items()])

In [ ]:
embed_dim=300
num_words=len(dictionary)+1
embedding_matrix=np.zeros([num_words,embed_dim])
for word, idx in dictionary.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [179]:


timesteps = 100

model = Sequential()
model.add(Embedding(num_words, embed_dim, weights = [embedding_matrix], input_length=window_size, trainable=True))
model.add(LSTM(timesteps, return_sequences=False, name='LSTM1'))
model.add(Dense(vocabSize, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop')


NameError: ignored

In [ ]:
model.summary

In [ ]:
hist = model.fit(X_train, y_train, batch_size=256, epochs=20, verbose=1, validation_data = (X_validation, y_validation))

In [ ]:
token = Tokenizer(num_words=30000,
                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,
                  split=' ', char_level=False, oov_token="UNK",
                  document_count=0)
token.fit_on_texts([book])

sequences = token.texts_to_sequences([book])

In [ ]:
def create_dataset(seq, window = 100):
    inputs = []
    outputs = []

    for i in range(len(seq)-window):
        inputs.append(seq[i:i+window])
        outputs.append(seq[i+1:i+window+1])
    return inputs, outputs

In [ ]:
x, y = create_dataset(sequences[0], window=100)

In [170]:
text_x=token.sequences_to_texts(x)
text_y=token.sequences_to_texts(y)

In [ ]:
print('x: ', text_x[1001])
print('y: ', text_y[1001])

x:  that you have a most clumsy and careless servant girl ” “my dear holmes ” said i “this is too much you would certainly have been burned had you lived a few centuries ago it is true that i had a country walk on thursday and came home in a dreadful mess but as i have changed my clothes i can’t imagine how you deduce it as to mary jane she is incorrigible and my wife has given her notice but there again i fail to see how you work it out ” he chuckled to himself and rubbed his
y:  you have a most clumsy and careless servant girl ” “my dear holmes ” said i “this is too much you would certainly have been burned had you lived a few centuries ago it is true that i had a country walk on thursday and came home in a dreadful mess but as i have changed my clothes i can’t imagine how you deduce it as to mary jane she is incorrigible and my wife has given her notice but there again i fail to see how you work it out ” he chuckled to himself and rubbed his long


In [ ]:
print('x: ', x[1001])
print('y: ', y[1001])

x:  [10, 14, 18, 6, 139, 4832, 3, 4833, 1690, 504, 8, 296, 244, 35, 8, 32, 7, 344, 16, 163, 107, 14, 53, 371, 18, 42, 1938, 21, 14, 952, 6, 174, 4834, 431, 11, 16, 399, 10, 7, 21, 6, 320, 825, 44, 3437, 3, 96, 268, 9, 6, 697, 4835, 28, 20, 7, 18, 1939, 17, 527, 7, 953, 400, 113, 14, 896, 11, 20, 4, 554, 4836, 39, 16, 4837, 3, 17, 245, 61, 361, 38, 1108, 28, 34, 203, 7, 1691, 4, 72, 113, 14, 141, 11, 54, 8, 12, 1480, 4, 184, 3, 2687, 15]
y:  [14, 18, 6, 139, 4832, 3, 4833, 1690, 504, 8, 296, 244, 35, 8, 32, 7, 344, 16, 163, 107, 14, 53, 371, 18, 42, 1938, 21, 14, 952, 6, 174, 4834, 431, 11, 16, 399, 10, 7, 21, 6, 320, 825, 44, 3437, 3, 96, 268, 9, 6, 697, 4835, 28, 20, 7, 18, 1939, 17, 527, 7, 953, 400, 113, 14, 896, 11, 20, 4, 554, 4836, 39, 16, 4837, 3, 17, 245, 61, 361, 38, 1108, 28, 34, 203, 7, 1691, 4, 72, 113, 14, 141, 11, 54, 8, 12, 1480, 4, 184, 3, 2687, 15, 148]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [ ]:
embed_dim=300
num_words=len(dictionary)+1
embedding_matrix=np.zeros([num_words,embed_dim])
for word, idx in dictionary.items():
  if idx <= num_words and word in embeddings_index:
    embedding_matrix[idx,:]=embeddings_index[word]

In [ ]:
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, MaxPooling1D, Dropout, Dense, Input, Concatenate,Dot,RepeatVector,TimeDistributed,Multiply,Lambda,Flatten, BatchNormalization, Activation, Bidirectional, LSTM, Reshape
import tensorflow.keras.backend as K
from tensorflow.keras.activations import softmax
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers